In [4]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path
from datetime import datetime
import re

In [5]:
s = requests.Session()

In [18]:
records_dir = Path('records')
records_dir.mkdir(exist_ok=True)

In [27]:
cookies = {"tapestry_session" : "eyJpdiI6IlV5V0Q0aDMzeTdKaVdtSFVMZldRclE9PSIsInZhbHVlIjoib0tqZ05hT09rOEttS3Y5MXBoYnorQnR2R1dlQ0VHNlkreTdTS3NUS1JwNktiekNRRUVRMzV1dXNPUXZsM0g4eHR2cnBWTG1UMFAwK2lZXC93SU0wNXU1eVwvNzZFUklxWk5PV3pwOXRmTjBcLzhSNEF2TXNFaDY1Q09FMHU0a3Y4WFgiLCJtYWMiOiIyYzJmNDZkNGYwZDhlNDc3YWMxZjk3NzE2ZmUzNmQ3ZTNhN2RkOTllYmZkOTVhNGM4N2Q3ZWQyMjg4NzY0YTkwIn0%3D"}

In [45]:
r = s.get("https://tapestryjournal.com/s/ridgefield-primary-school/observations?page_size=100", 
          cookies=cookies)
assert r.status_code == 200
soup = BeautifulSoup(r.content, 'html.parser')

In [11]:
def parse_added_str(added_str):
    dt = added_str[:20]
    reformated_dt = datetime.strptime(dt, '%d %b %Y %I:%M %p').strftime('%Y%m%d_%H%M')
    auth = added_str[24:]
    return dt, reformated_dt, auth

In [12]:
def find_img_extension(url):
    qm_pos = url.find('?')
    if qm_pos < 0:
        raise ValueError('ERROR: could not find parameters')
    extention_start = url[:qm_pos].rfind('.')
    if extention_start < 0:
        raise ValueError("ERROR: could not find extention")
    return url[extention_start:qm_pos]
              
find_img_extension('https://tapestryjournal.com/xs/pages/s_42402/p_07-2022/t_o_15757_fxnvd6qqjxssq36w7a37vk454xzz2gxs.jpg?s=42402&u=290&Expires=1658360652&Signature=Y67W~D5EzAtQmXMvmO7iBMn30AMmlJ5X9JO4YtP0gIze3y3hYdn3xZ0gvMQO65X30T6wnqgt~naAlaqQfO3WGHEul45AWE9RhELjLEeunQ2s8SGU7y-TlbbrLMv4z5h-cgJH-1iWYjJqpjg9yOd0Bm2dO~dZr7LGUI~faF1mVGKfCVND0gg5hBp7noaFIEbxvTanHyfdLZdOTp0HFHyQD6QDyxJpHxt-6gWYEKVSEtNvqpysDUXVCcdmZzy0up41KTxACEhXW6CYQ7aGXHFA9cWVXRsOM8CqaqUMsSSyGIhSUGaxnLvetf-878Ma2m3fTm1V-zJvWV2J8YBMLmuQbA__&Key-Pair-Id=APKAJUSDRV55TOQKSATA')

'.jpg'

In [9]:
def get_all_media(media_blocks, proceeded_num=0):
    print(f'{proceeded_num = }')
    missing = []
    for mb_i, mb in enumerate(media_blocks):
        if mb_i < proceeded_num:
            continue

        time_added = mb.find('h4').findNext('div').text.strip()
        orig_dt, formatted_dt, author = parse_added_str(time_added)
        print('\n', orig_dt, formatted_dt, author)

        album_url_tag = mb.find(lambda tag:tag.name=="a" and "Download Media" in tag.text)
        if not album_url_tag:
            print(f'{mb_i}: NO MEDIA')
            proceeded_num += 1
            continue
        album_link = album_url_tag['href']
        print(f'{album_link = }')

        r_album = s.get(album_link)
        soup_album = BeautifulSoup(r_album.content, 'html.parser')
        photo_tags = soup_album.find_all('a')
        print(f'{len(photo_tags) = }')

        for i, photo_tag in enumerate(photo_tags):
            media_url = photo_tag['href']

            photo_req = s.get(media_url, stream=True)

            if photo_req.status_code == 200:
                if "content-disposition" in photo_req.headers:
                    continue
                    cd = photo_req.headers["content-disposition"]
                    cd_attr = {k: v for k, v in 
                         filter(lambda x: len(x) > 1, map(lambda x: x.strip(';').split('='), cd.split()))}
                    orig_filename = cd_attr["filename"]
                    filename = f'{formatted_dt}_{i:02d}_{orig_filename}'
                elif photo_tag.has_attr('download'):
                    filename = f'{formatted_dt}_{i:02d}_{photo_tag["download"]}'
                else:
                    raise ValueError('ERROR: unknown attachment type')
                
                print(f'{filename = }')

                with open(records_dir / filename, 'wb') as f:
                    f.write(photo_req.content)
            else:
                missing.append((mb_i, time_added, i))
                raise ConnectionError

        proceeded_num += 1

    print(f'{proceeded_num = }')
    print(f'{missing = }')
    
    return proceeded_num

In [43]:
def get_all_memos(media_blocks):
    all_memos = []
    raw_memos = []
    
    for mb in media_blocks[::-1]:
        cur_memo = []
        cur_raw_memo = ''
        
        title = mb.find('h4').find('a').text.strip()
        cur_memo.append(f'## {title}')
        cur_raw_memo += f'## {title}\n'
        
        time_added = mb.find('h4').findNext('div').text.strip()
        cur_memo.append(f'### {time_added}')
        cur_raw_memo += f'### {time_added}\n'
        
        memo_url = mb.find('h4').find('a')['href']
        req_memo = s.get(memo_url)
        memo_soup = BeautifulSoup(req_memo.content, 'html.parser')
        
        notes_tag = memo_soup.find('div', {"class": "page-note"})
        if notes_tag:
            cur_raw_memo += f'{notes_tag}'

            paragraphs = list(map(lambda x: f'{x.text.strip()}  ', notes_tag.find_all('p')))
            cur_memo.extend(paragraphs)

        all_memos.append('\n'.join(cur_memo))
        raw_memos.append(cur_raw_memo)
    
    return '\n\n'.join(all_memos), '\n\n'.join(raw_memos)

In [46]:
media_blocks = soup.find_all('div', {"class": "media"})
len(media_blocks)

36

In [11]:
proceeded_num = 0

In [13]:
proceeded_num = get_all_media(media_blocks, proceeded_num)

In [47]:
memos, raw_memos = get_all_memos(media_blocks)

with open(records_dir / 'memos.md', 'w', encoding='utf-8') as f:
    f.write(memos)
    
with open(records_dir / 'memos_raw.txt', 'w', encoding='utf-8') as f:
    f.write(raw_memos)

In [32]:
r_common = s.get("https://tapestryjournal.com/s/ridgefield-primary-school/memos?page_size=100")
assert r_common.status_code == 200
soup_common = BeautifulSoup(r_common.content, 'html.parser')

In [33]:
common_media_blocks = soup_common.find_all('div', {"class": "media"})
len(common_media_blocks)

18

In [173]:
proceeded_num = 0

In [178]:
proceeded_num = get_all_media(common_media_blocks, proceeded_num)

proceeded_num = 0

 08 Jul 2022 04:04 PM 20220708_1604 Kay Barber
album_link = 'https://tapestryjournal.com/s/ridgefield-primary-school/pages/media-download-modal/8898'
photo_tags = [<a href="https://tapestryjournal.com/s/ridgefield-primary-school/pages/media-download?page=8898&amp;media=15839">
<img height="90" src="https://tapestryjournal.com/1s/pages/s_42402/p_07-2022/t_o_15839_h7dz5fm3vccadgr4zxz5c5tkx09f061s.jpg?s=42402&amp;u=290&amp;Expires=1658375079&amp;Signature=dPE6BMpbskfEyvVPk05K820jo9-BWp0FiXLc9zpk9a5n25VhyP~~Wla2n6HmC3gj962EPB-AcG3OROlWZ59iqKE~YVbLZk1jbiN3bUlqVdCBg19Hhw6UH3Kh0RWnxcnLzJBXqADrNvxtadhdFPpQ~cELqrRBjS9sI2G3tzItdNTlaLGPHbiaH3rFpy2~M24bwVMxoIuKRuXVZ1zY7sbyO557FmqMXOyKsvRfozgVUwblNCnpyopaJQ8TTIqWtHRxi4ocpFH9jcIveHFiuWbe-lYPu4g~vjH7P9AwE9U5Mte0L5G8Lrklpxf5cSpSOjAVm40oLLrMKvfYYMyw7qtbtQ__&amp;Key-Pair-Id=APKAJUSDRV55TOQKSATA" width="90"/>
<i class="fa fa-camera fa-3x" title="Photo: moving up day.png"></i>
</a>, <a class="btn btn-default btn-sm page-media-document-b

In [44]:
com_memos, raw_com_memos = get_all_memos(common_media_blocks)

with open(records_dir / 'com_memos.md', 'w', encoding='utf-8') as f:
    f.write(com_memos)
    
with open(records_dir / 'com_memos_raw.txt', 'w', encoding='utf-8') as f:
    f.write(raw_com_memos)